In [1]:
# %pip install pandas_datareader
# %conda install - c conda-forge pystan
# %conda update - n base - c defaults conda
# %conda install -c conda-forge prophet

In [11]:
import pandas_datareader.data as web

#期間指定
st = '2016/01/01'
ed = '2022/05/01'
st_train = '2016/01/01'
ed_train = '2022/03/31'
st_test = ed_train
ed_test = '2022/04/30'

#期間を指定し、NTTデータの株価を取得
stooq = web.DataReader(['9613.T'], 'yahoo',start=st,end=ed)

In [12]:
stooq.head()

Attributes,Adj Close,Close,High,Low,Open,Volume
Symbols,9613.T,9613.T,9613.T,9613.T,9613.T,9613.T
Date,,,,,,
2016-01-04,1058.973999,1146.0,1174.0,1142.0,1160.0,4254000.0
2016-01-05,1066.366211,1154.0,1164.0,1140.0,1146.0,3219000.0
2016-01-06,1051.581421,1138.0,1158.0,1134.0,1148.0,3997500.0
2016-01-07,1046.036987,1132.0,1150.0,1128.0,1140.0,3611000.0
2016-01-08,1055.277466,1142.0,1158.0,1110.0,1112.0,9022500.0


In [13]:
import numpy as np

#####################学習データ
df = stooq[st_train:ed_train]
df = df.dropna(how='any')

#対数変換する場合
#df['Adj Close'] = np.log(df['Adj Close'])

df = df.reset_index()
df[['ds','y']] = df[['Date' ,'Adj Close']] #カラム名を変更
df = df[['ds','y']] #余分なカラムを削除


#####################テスト（正解）データ
df_actual = stooq[st_test:ed_test]
df_actual = df_actual.dropna(how='any')

#対数変換する場合
#df_actual ['Adj Close'] = np.log(df_actual ['Adj Close'])

df_actual = df_actual .reset_index()
df_actual [['ds','y']] = df_actual [['Date' ,'Adj Close']] #カラム名を変更
df_actual = df_actual [['ds','y']] #余分なカラムを削除


#####################オリジナルデータ（学習データ+テストデータ）（可視化用）
stooq = stooq.dropna(how='any')

#対数変換する場合
#stooq['Adj Close'] = np.log(stooq['Adj Close'])

stooq = stooq.reset_index()
stooq[['ds','y']] = stooq[['Date' ,'Adj Close']] #カラム名を変更
stooq = stooq[['ds','y']] #余分なカラムを削除

In [14]:
stooq.head()

Attributes,ds,y
Symbols,,
0,2016-01-04,1058.973999
1,2016-01-05,1066.366211
2,2016-01-06,1051.581421
3,2016-01-07,1046.036987
4,2016-01-08,1055.277466


In [15]:
from fbprophet import Prophet, plot

# 予測日数（4〜5月)
period = 61
m = Prophet()
m.fit(df)

future = m.make_future_dataframe(periods=period)
forecast = m.predict(future)


# 予測結果の可視化
plot.plot_plotly(m, forecast)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [16]:
plot.plot_components_plotly(m, forecast)


In [17]:
from sklearn.metrics import mean_squared_log_error
import pandas as pd


# stooq.columns = stooq.columns.droplevel(1)
ans = pd.merge(forecast, stooq, on="ds", how='inner')
ans.set_index("ds",inplace=True)


##########RMSLE
y_pred = ans[st_test:ed_test]["yhat"]
y_true = ans[st_test:ed_test]["y"]

#対数変換した場合
#y_pred = np.exp(y_pred)
#y_true = np.exp(y_true)

RMSLE=np.sqrt(mean_squared_log_error (y_true, y_pred))
print("RMSLE:",RMSLE)



##########予測範囲に入っているかどうか
y_pred_lower = ans[st_test:ed_test]["yhat_lower"]
y_pred_upper = ans[st_test:ed_test]["yhat_upper"]
ans_judge = ans[st_test:ed_test].copy()

#対数変換した場合
#y_pred_lower = np.exp(y_pred_lower)
#y_pred_upper = np.exp(y_pred_upper)

cnt = 0
for i in ans_judge.index:
   if y_pred_lower[str(i)] <= y_true[str(i)] <= y_pred_upper[str(i)]:
      cnt += 1
print("予測数：",len(ans_judge.index),"範囲内：",cnt)

/Users/hiroto/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning:

merging between different levels can give an unintended result (1 levels on the left,2 on the right)



KeyError: 'y'

# 一年分の予測

# 1年の空のデータフレームを用意


In [ ]:
future_df = m.make_future_dataframe(365)
future_df.tail()


,ds
1906,2023-03-27
1907,2023-03-28
1908,2023-03-29
1909,2023-03-30
1910,2023-03-31


In [ ]:
# 予測の作成。勝手に埋めてくれる。
forecast_df = m.predict(future_df)
forecast_df[["ds", "yhat"]]


,ds,yhat
0,2016-01-04,1136.253155
1,2016-01-05,1136.141134
2,2016-01-06,1136.222753
3,2016-01-07,1134.899209
4,2016-01-08,1133.028473
...,...,...
1906,2023-03-27,3165.925151
1907,2023-03-28,3168.378814
1908,2023-03-29,3171.063634
1909,2023-03-30,3172.399469


In [ ]:
# plotlyでプロット
from fbprophet import plot
plot.plot_plotly(m, forecast_df)


In [ ]:
# 要素別の予測値プロット
plot.plot_components_plotly(m, forecast_df)
